In [1]:
import geopandas as gpd
import rasterio
from rasterio import features

ImportError: DLL load failed: The specified module could not be found.

In [2]:
import rasterio


ImportError: DLL load failed: The specified module could not be found.

Set up your filenames

In [ ]:
shp_fn = 'cb_2013_us_county_20m.shp'
rst_fn = 'template_raster.tif'
out_fn = './rasterized.tif'

Open the file with GeoPANDAS read_file

In [ ]:
counties = gpd.read_file(shp_fn)

Add the new column (as in your above code)

In [ ]:
for i in range (len(counties)):
    LSAD = counties.at[i,'LSAD']
    if LSAD == 00 :
        counties['LSAD_NUM'] == 'A'
    elif LSAD == 03 :
        counties['LSAD_NUM'] == 'B'
    elif LSAD == 04 :
        counties['LSAD_NUM'] == 'C'
    elif LSAD == 05 :
        counties['LSAD_NUM'] == 'D'
    elif LSAD == 06 :
        counties['LSAD_NUM'] == 'E'
    elif LSAD == 13 :
        counties['LSAD_NUM'] == 'F'
    elif LSAD == 15 :
        counties['LSAD_NUM'] == 'G'  
    elif LSAD == 25 :
        counties['LSAD_NUM'] == 'I'          
    else :
        counties['LSAD_NUM'] == 'NA'


Open the raster file you want to use as a template for feature burning using rasterio


In [ ]:
rst = rasterio.open(rst_fn)


copy and update the metadata from the input raster for the output


In [ ]:
meta = rst.meta.copy()
meta.update(compress='lzw')

Now burn the features into the raster and write it out


In [ ]:
with rasterio.open(out_fn, 'w', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(counties.geometry, counties.LSAD_NUM))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)



The overall idea is to create an iterable containing tuples of (geometry, value), where the geometry is a shapely geometry and the value is what you want to burn into the raster at that geometry's location. Both Fiona and GeoPANDAS use shapely geometries so you are in luck there. In this example a generator is used to iterate through the (geometry,value) pairs which were extracted from the GeoDataFrame and joined together using zip().